# Question answering using embeddings-based search

<u><b>Full procedure<b><u>

1. Prepare search data (once per document)

- Collect: We'll download a few hundred bug reports
- Chunk: Documents are split into short, mostly self-contained sections to be embedded
- Embed: Each section is embedded with the OpenAI API
- Store: Embeddings are saved (for large datasets, use a vector database)

2. Search (once per query)

- Given a user question, generate an embedding for the query from the OpenAI API
- Using the embeddings, rank the text sections by relevance to the query

3. Ask (once per query)

- Insert the question and the most relevant sections into a message to GPT
- Return GPT's answer

4. Answers
- Some example questions are made to the system
- The bug reports in which the answers are found can also be retrieved

### Imports and Read Data

In [1]:
import ast  
from scipy import spatial  
import pandas as pd 

from openai import OpenAI
import tiktoken 

# Create a list of models 
GPT_MODELS = ["gpt-4o", "gpt-4o-mini"]
# Embedding models
EMBEDDING_MODEL = "text-embedding-3-small"

client = OpenAI()

In [2]:
# Retrieve the bug reports from the request_data notebook
data_path = "bug_reports_mozilla_firefox_resolved_fixed_comments_embeddings.csv"

# The embeddings are already stored in the CSV, and need to be transformed from strings to lists
data = pd.read_csv(data_path)
data["Embeddings"] = data["Embeddings"].apply(ast.literal_eval)

### Search

Now we'll define a search function that:

- Takes a user query and a dataframe with text & embedding columns
- Embeds the user query with the OpenAI API
- Uses distance between query embedding and text embeddings to rank the texts
- Returns two lists:
1. The top N texts, ranked by relevance
2. Their corresponding relevance score

In [3]:
# Search function
def strings_ranked_by_relatedness(
    query: str,
    data: pd.DataFrame,
    relatedness_fn = lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100
) -> tuple[list[str], list[float]]:
    
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = client.embeddings.create(
        model = EMBEDDING_MODEL,
        input = query,
    )
    query_embedding = query_embedding_response.data[0].embedding
    strings_and_relatednesses = [
        (row["Concat"], relatedness_fn(query_embedding, row["Embeddings"]))
        for i, row in data.iterrows()
    ]
    strings_and_relatednesses.sort(key = lambda x: x[1], reverse = True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    
    return strings[:top_n], relatednesses[:top_n]

In [5]:
# Examples
strings, relatednesses = strings_ranked_by_relatedness("User Interface (UI)", data, top_n = 5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness = :.3f}")
    display(string)
    print("\n")

relatedness = 0.510


'summary provide access fxa device management ui description this meta bug track bug block on u get manage device ui while ui almost certainly live on server need exist way client can deliver require ux'



relatedness = 0.473


'summary web base stub install ui description current stub install user interface code highly fragile error prone difficult modify difficult read the number engineer available work on worrying small extremely difficult bring new engineer code specialized replace current implement web page render native web view make modify ui visual design much fast easy specialist engineer would no long need make change change can made frequent less risk it would also remove lot limit current exist on ui element may use can custom note current web base mean use html css not actually downloade ui element web runtime'



relatedness = 0.443


'summary initial payment request basic credit card add edit page description share logic ui preference management ui'



relatedness = 0.434


'summary crash user interface update description track bug the current crash user interface problem need fixed the problem fix list track bug this bug cover implement user interface change this mock new user interface'



relatedness = 0.425


'summary login identify secure internal ui page description login brand logo address bar identity box show secure internal ui page'

### Ask

With the search function above, we can now automatically retrieve relevant knowledge and insert it into messages to GPT. Below, we define a function ask that:

- Takes a user query
- Searches for text relevant to the query
- Stuffs that text into a message for GPT
- Sends the message to GPT
- Returns GPT's answer

In [6]:
def num_tokens(text: str, model: str = GPT_MODELS[0]) -> int:
    
    """Return the number of tokens in a string."""
    
    encoding = tiktoken.encoding_for_model(model)
    
    return len(encoding.encode(text))

In [7]:
def query_message(
    query: str,
    data: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    
    strings, relatednesses = strings_ranked_by_relatedness(query, data)
    introduction = 'Use the bug reports from Bugzilla to answer the subsequent question. If the answer cannot be found, write "I could not find an answer."'
    question = f"\n\nQuestion: {query}"
    message = introduction
    
    for string in strings:
        next_bug = f'\n\nBug Report Summary:\n"""\n{string}\n"""'
        if (
            num_tokens(message + next_bug + question, model = model)
            > token_budget
        ):
            break
        else:
            message += next_bug
            
    return message + question

In [8]:
def ask(
    query: str,
    data: pd.DataFrame = data,
    model: str = GPT_MODELS[0],
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    
    message = query_message(query, data, model = model, token_budget = token_budget)
    
    if print_message:
        print(message)
        
    messages = [
        {"role": "system", "content": "You answer questions about the Bugzilla bug reports"},
        {"role": "user", "content": message},
    ]
    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0
    )
    response_message = response.choices[0].message.content
    
    return response_message

### Answers

In [11]:
print(ask('What is the most common issue in these bug reports?'))

The most common issue in these bug reports appears to be related to crashes and performance problems. Several reports mention crashes, such as "crash description this bug crash report" and "crash on machine code landed bug maybe fission related crash report." Additionally, there are reports of performance issues, such as "activitysteam screenshott appear hang main process multiple second apparently after visiting link" and "investigate decide manage alpenglow performance." These types of issues seem to be a recurring theme across multiple reports.


In [12]:
print(ask('What are the issues related to User Interface (UI) in these bug reports?'))

The bug reports related to User Interface (UI) issues include:

1. Crash in the user interface during updates and changes (first bug report).
2. Applying UX design to translate popups (second bug report).
3. Improving UI for page block policy (fifth bug report).
4. Removing pre-Firefox UI migration steps (sixth bug report).
5. Fixing CustomizableUI failures in showing the Proton toolbar (tenth bug report).
6. Simplifying PanelUI show and adding tests for opening and closing the main menu (eleventh bug report).
7. Providing access to FxA device management UI (thirteenth bug report).
8. Updating preference UI for Firefox to suggest best match (fifteenth bug report).
9. Improving toolbar customization on OS (seventeenth bug report).
10. Removing old UI migration steps (eighteenth bug report).
11. Updating translate popup icon with UX recommended treatment (nineteenth bug report).
12. Addressing incorrect display of the "Share Select Device" button (twenty-first bug report).
13. Hiding the

In [26]:
print(ask('What are the issues related to User Interface (UI) in these bug reports?', print_message = True))

Use the bug reports from Bugzilla to answer the subsequent question. If the answer cannot be found, write "I could not find an answer."

Bug Report Summary:
"""
summary crash user interface update description track bug the current crash user interface problem need fixed the problem fix list track bug this bug cover implement user interface change this mock new user interface
"""

Bug Report Summary:
"""
summary apply ux design translate popup description this bug track follow work strict apply feedback on design popup ui see bug initial prototype
"""

Bug Report Summary:
"""
summary bug followup apply searchbar add engine overflow menu css change window linux description this bug initially create clone bug
"""

Bug Report Summary:
"""
summary bugzilla component uriloader incorrect mot yaml description the bugzilla component uriloader state core document navigation state correct component firefox file handle
"""

Bug Report Summary:
"""
summary improve ui page block policy description t

In [14]:
print(ask("About what the issues related to Firefox web browser talk about?"))

The issues related to the Firefox web browser discussed in the bug reports include:

1. New panel features and design documentation.
2. Security issues and emergency bulletins.
3. Migration from 32-bit to 64-bit Firefox versions.
4. Improvements in tab browsing.
5. Crashes when accessing download options.
6. New profile welcome messages and user engagement.
7. Localization information for Firefox updates.
8. Vulnerabilities in Firefox add-ons and potential exploits.
9. Updates and redirection for Firefox view code.
10. Design updates for welcome back pages.
11. Availability of Firefox Labs outside the Nightly train.
12. Issues with bookmarks not leading to themes or extensions.
13. Branding changes for Firefox Beta.
14. Updates to default theme information.
15. Transition plans for enabling Firefox Translate.
16. Issues with clean branch builds and first page loading.
17. Enabling Firefox View Next by default in Nightly builds.
18. UI issues with URL bar and search dropdowns.
19. Probl

In [30]:
print(ask("What are the most common software components affected?"))

Based on the provided bug report summaries, the most common software components affected include:

1. **XUL (XML User Interface Language)** - Mentioned in several reports related to crashes and layout issues.
2. **NSI (Nullsoft Scriptable Install System)** - Referenced in installation-related bugs.
3. **IPC (Inter-Process Communication)** - Involved in crash reports.
4. **JavaScript Modules (JSM)** - Referenced in performance and startup issues.
5. **Lodash** - Mentioned in reports about vulnerabilities and dependency updates.
6. **Telemetry** - Involved in reports about message impression and sync preferences.
7. **Sync Engine** - Referenced in reports about sync errors and preferences.

These components appear frequently across different bug reports, indicating they are commonly affected in various issues.


In [31]:
print(ask("What are the most common features affected?"))

I could not find an answer.


In [29]:
print(ask("What words are used the most to describe the issues?"))

The words most frequently used to describe the issues in the Bugzilla bug reports are:

1. Description
2. Bug
3. Issue
4. Summary
5. Fix
6. Update
7. String
8. Preference
9. Telemetry
10. Improve

These words are commonly associated with the nature of bug reports, which typically involve describing issues, proposing fixes, and updating or improving software components.
